In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import h5py
import keras.backend as K
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Activation 
from keras import optimizers, layers, callbacks
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


## Adaptation cost (CB9,CB10,CB11,CB12)

In [2]:
x_1 = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Separated dataset_ML/a1_train_scale.csv', 
                    delimiter = ',', usecols = (19,20,21,22), skip_header=2)
y_1o = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Separated dataset_ML/a1_train_scale.csv', 
                   delimiter =',', usecols = (29), skip_header=2).reshape (54,1)
x_2 = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Separated dataset_ML/a1_cv_scale.csv', 
                    delimiter = ',', usecols = (19,20,21,22), skip_header=1)
y_2o = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Separated dataset_ML/a1_cv_scale.csv',
                      delimiter =',', usecols = (29),skip_header=1).reshape (14,1)

In [3]:
enc = OneHotEncoder ()
enc.fit (y_1o)
y_1 = enc.transform(y_1o).toarray()
y_2 = enc.transform(y_2o).toarray()

In [4]:
y_whole = np.vstack ((y_1, y_2)).astype(np.float32)
x_whole = np.vstack ((x_1, x_2)).astype(np.float32)
x_traind = x_1
x_test = x_2
y_traind = y_1
y_test = y_2
x_train, x_cv, y_train, y_cv = train_test_split (x_1,y_1,test_size=0.13)

In [5]:
num_classes = 4

model = Sequential ()
model.add (Dense (units = 29, input_dim = 4))
model.add (layers.Activation ('tanh'))
model.add (Dense (units = 29))
model.add (layers.Activation ('tanh'))
model.add (Dense (units = 29))
#model.add (layers.Activation ('tanh'))
#model.add (Dense (units = 16))
model.add (Dense(num_classes, activation = 'softmax'))

#sgd = optimizer.SGD (lr=0.01, momentum = 0, decay =0, nestrov = False)
model.compile (loss = 'binary_crossentropy', optimizer = 'sgd', 
               metrics=['binary_accuracy'])

History = model.fit(x_train, y_train,
                    batch_size=50,
                    epochs=50,
                    verbose=1, validation_data=(x_cv, y_cv))

print(model.evaluate (x_test, y_test, batch_size = 50, verbose =1))

#help (model.evaluate)
#this is the output units

y_pred_train = model.predict (x_train)
y_pred_cv = model.predict (x_cv)
y_pred_test = model.predict (x_test)
y_pred_whole = np.vstack ((y_pred_train, y_pred_cv, y_pred_test)).astype(np.float32)

Train on 46 samples, validate on 8 samples
Epoch 1/50
46/46 [==============================] - ETA: 0s - loss: 0.6358 - binary_accuracy: 0.7500 - val_loss: 0.5288 - val_binary_accuracy: 0.7500
Epoch 2/50
46/46 [==============================] - ETA: 0s - loss: 0.6338 - binary_accuracy: 0.7500 - val_loss: 0.5294 - val_binary_accuracy: 0.7500
Epoch 3/50
46/46 [==============================] - ETA: 0s - loss: 0.6318 - binary_accuracy: 0.7500 - val_loss: 0.5300 - val_binary_accuracy: 0.7500
Epoch 4/50
46/46 [==============================] - ETA: 0s - loss: 0.6298 - binary_accuracy: 0.7500 - val_loss: 0.5306 - val_binary_accuracy: 0.7500
Epoch 5/50
46/46 [==============================] - ETA: 0s - loss: 0.6279 - binary_accuracy: 0.7500 - val_loss: 0.5313 - val_binary_accuracy: 0.7500
Epoch 6/50
46/46 [==============================] - ETA: 0s - loss: 0.6260 - binary_accuracy: 0.7500 - val_loss: 0.5319 - val_binary_accuracy: 0.7500
Epoch 7/50
46/46 [==============================] - ETA: 

### t(x) for labelling

In [6]:
#save model to json, weights to HDF5
model_json = model.to_json ()
with open ('model_lod_adapt_cost_a.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('model_lod_adapt_cost_a.h5')
print ('Saved model to disk')

Saved model to disk


In [5]:
#load json and create model
json_file = open('model_lod_adapt_cost_a.json', 'r')
loaded_model_json = json_file.read ()
json_file.close ()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights('model_lod_adapt_cost_a.h5')

In [6]:
loaded_model.compile(loss = 'binary_crossentropy', optimizer = 'sgd', 
               metrics=['binary_accuracy'])
print(loaded_model.evaluate (x_test, y_test, batch_size = 20, verbose =1))
y_pred_test = loaded_model.predict (x_test)


i = 0;
j = 0;
m = np.zeros((14,4));
for i in range (14):
    max = 0.0000001;
    for j in range (4):
        if y_pred_test.item ((i,j))>=max:
            max = y_pred_test.item ((i,j))
    for j in range (4):
        if y_pred_test.item ((i,j)) ==max:
            np.put(m, [4*i+j], [1])

i = 0;
j = 0;
selected_LOD = np.zeros((14,1));
for i in range (14):
    for j in range (4):
        if m.item((i,j))>0:
            selected_LOD_1 = m.item((i,j))*y_pred_test.item((i,j));
            np.put(selected_LOD, [i], [selected_LOD_1])

i = 0;
j = 0;
cf = np.zeros((14,4))
for i in range (14):
    for j in range (4):
        cf_1 = y_pred_test.item((i,j))-selected_LOD.item((i,0));
        cf_2 = 10*cf_1;
        np.put(cf, [4*i+j], [cf_2])
print (cf)
np.savetxt('y_cost_LODadapt_cost.csv', cf, delimiter=',') 

14/14 [==============================] - ETA: 0s
[0.57749545574188232, 0.75]
[[-0.57835236 -1.28792852  0.         -0.28383076]
 [-0.97886071 -1.18554294  0.         -0.15913606]
 [-0.06279856  0.         -1.14132196 -0.53873107]
 [-0.20969063 -0.07426232 -0.23439199  0.        ]
 [-1.17324546 -0.49163103 -0.40204823  0.        ]
 [-0.12936503  0.         -1.37849376 -0.7535553 ]
 [-0.31342298  0.         -0.4436633  -0.05060077]
 [-0.76307788 -0.72112143 -0.14950991  0.        ]
 [-1.07603207 -1.23314038 -0.25041759  0.        ]
 [-0.59000239 -1.01625159  0.         -0.23620605]
 [-0.86731613 -0.9713757  -0.31519681  0.        ]
 [-0.43058127 -0.30294806 -0.3461647   0.        ]
 [-1.35407656 -1.68226808  0.         -0.36144286]
 [-0.80324158 -0.78267679 -0.82639515  0.        ]]
